In [4]:
import pandas as pd
import numpy as np
import scipy.stats as sct
import seaborn as sns
from matplotlib import pyplot as plt
import os

Clique [dataset_sp_limpo.csv ](https://drive.google.com/file/d/1dhhFIM1qZILh0LONo1msRgLuEAO3aThy/view?usp=sharing) no diretorio dataset

In [10]:
df = pd.read_csv(os.path.join("dataset", "dataset_sp_limpo.csv"), index_col=0)
df = df[df.cs_sexo != "INDEFINIDO"]

# Testes paramétricos

## Teste para verificar se a frequência de gênero sexual entres os óbitos de COVID-19 no dataset é compatível com a frequência na população 

Óbito por gênero sexual: Mulheres - 45%, Homens - 55% | [Fonte](https://www.em.com.br/app/noticia/gerais/2020/09/09/interna_gerais,1183974/covid-19-atinge-principalmente-mulheres-mas-mata-mais-homens-ufmg.shtml)

H0: frequências observadas = frequências esperadas.

H1: as frequências observadas ≠ frequências esperadas.

Alfa: 0.05

In [7]:
frequencia_observada = df[df.obito == 1].cs_sexo.value_counts(normalize=True) * 100
frequencia_esperada = pd.Series({"MASCULINO": 55, "FEMININO": 45}) 
sct.chisquare(f_obs=frequencia_observada, f_exp=frequencia_esperada, axis=None)

Power_divergenceResult(statistic=0.3007274015016478, pvalue=0.5834267643596402)

p-valor > alfa, portanto aceita-se H0

## Teste para verificar se a idade média da pessoas que faleceram de COVID-19 é superior a idade média das pessoas que não faleceram